In [1]:
from neighbor_server import *
from social_graph import *
from shortest_paths import *

In [2]:
sg = SocialGraph("train")

Initializing check ins
Loading from Training Pickle File
Initializing network


In [3]:
from __future__ import print_function

In [4]:
'''
when you add neighbors into the graph
their g(source) = dist of last check in from source
and their h(n) = dist of last check in from end

g is cost of geographic movement from source
h becomes hueristic to choose the closest neighbor to goal

to throttle the API,
pick the top k nodes at each step that have f = g + h the minimum
rather than putting all nodes in the priority queue

otherwise put all the nodes in the queue, hoping for hueristic to take care of the problem
also maintain the got_neighbors_for set so that multiple test cases can use the same graph
'''

'\nwhen you add neighbors into the graph\ntheir g(source) = dist of last check in from source\nand their h(n) = dist of last check in from end\n\ng is cost of geographic movement from source\nh becomes hueristic to choose the closest neighbor to goal\n\nto throttle the API,\npick the top k nodes at each step that have f = g + h the minimum\nrather than putting all nodes in the priority queue\n\notherwise put all the nodes in the queue, hoping for hueristic to take care of the problem\nalso maintain the got_neighbors_for set so that multiple test cases can use the same graph\n'

In [4]:
shortest_path, path_length, num_queries, got_neighbors_for = astar_distance_based(sg, 5555, 2222, set(), "train", g_fn=dists.unweighted, h_fn=dists.l2)

Destination Reached
Number of Queries made: 184


In [5]:
shortest_path

[5555, 648, 30447, 2222]

In [ ]:
shortest_path, path_length, num_queries, got_neighbors_for = astar_distance_based(sg, 5555, 2222, set(), "train", g_fn=dists.unweighted, h_fn=dists.l2)

In [13]:
shortest_path, path_length, num_queries, got_neighbors_for = astar_distance_based(sg, 939, 401, set(), "train", g_fn=dists.unweighted, h_fn=dists.manhattan)

Destination Reached


In [14]:
shortest_path

[939, 5294, 27493, 10278, 401]

In [15]:
path_length

4

In [18]:
shortest_path, path_length, got_neighbors_for = astar_distance_based(sg, 5555, 2222, set(), "train", g_fn=dists.manhattan, h_fn=dists.manhattan)

Destination Reached


In [19]:
shortest_path

[5555, 8, 5187, 19390, 2222]

In [4]:
# path length
task1_combs = [(dists.unweighted, dists.manhattan), (dists.unweighted, dists.l2)]
# geographic length
task2_combs = [(dists.manhattan, dists.manhattan), (dists.l2, dists.l2)]

In [6]:
source = 5555
dest = 2222
for g_func, h_func in task1_combs:
    shortest_path, path_length, num_queries, got_neighbors_for = astar_distance_based(
        sg, source, dest, set(), "train", 
        g_fn=g_func, h_fn=h_func
    )
    print(shortest_path, path_length)

Destination Reached
Number of Queries made: 146
[5555, 648, 27809, 1734, 7229, 2222] 5
Destination Reached
Number of Queries made: 184
[5555, 648, 30447, 2222] 3


In [9]:
source = 42568
dest = 8195
for g_func, h_func in task1_combs:
    shortest_path, path_length, num_queries, got_neighbors_for = astar_distance_based(
        sg, source, dest, set(), "train", 
        g_fn=g_func, h_fn=h_func
    )
    print(shortest_path, path_length)

Destination Reached
Number of Queries made: 8259
[42568, 24952, 41, 27389, 2415, 10029, 8195] 6
Destination Reached
Number of Queries made: 7089
[42568, 24952, 41, 27389, 2415, 10029, 8195] 6


In [38]:
# returns the path in reverse
def reconstructed_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path

def get_path(node, neighbor, came_from_source, came_from_dest):
    # process path
    source_path = [p for p in reversed(reconstructed_path(came_from_source, node))]
    dest_path = reconstructed_path(came_from_dest, neighbor)
    path = source_path + dest_path
    return path
    
def expand_frontier(sg, node, start_frontier, target_frontier, 
                    fq, g_source, g_dest, came_from_source, came_from_dest,
                    closed_set, num_queries, 
                    g_fn, h_fn, opt="train"):
    
    if node not in closed_set:
        num_queries.append(1)

    neighbors = sg.get_neighbors(node)
    for neighbor in neighbors:
        if neighbor in target_frontier:
            # path found
            print("Path found at:", node, neighbor)
            path = get_path(node, neighbor, came_from_source, came_from_dest)
            path_length = g_source[node] + sg.get_dist(node, neighbor, g_fn) + g_dest[neighbor]
            return (path, path_length)
        else:
            # compute h
            neighbor_h = sys.maxsize
            for tn in target_frontier:
                curr_h = g_dest[tn] + sg.get_dist(neighbor, tn, h_fn)
                neighbor_h = min(neighbor_h, curr_h)
            
            # for every neighbor, compute f
            tentative_g = g_source[node] + sg.get_dist(node, neighbor, g_fn)
            if neighbor not in g_source or tentative_g < g_source[neighbor]:
                g_source[neighbor] = tentative_g
                f_score = g_source[neighbor] + neighbor_h
                came_from_source[neighbor] = node
                fq.put((f_score, neighbor))
    # add to start frontier
    for neighbor in neighbors:
        start_frontier.add(neighbor)
    
    return (None, None)

def frontier_search(sg, source, dest, got_neighbors_for, testcase_id=1, g_fn=dists.unweighted, h_fn=dists.manhattan):
    source_frontier = { source }
    closed = set()
    dest_frontier = { dest }
    num_queries = 0
    came_from_source = {}
    came_from_dest = {}
    
    g_source = {}
    g_dest = {}
    for node in sg.node_checkins:
        g_source[node] = sys.maxint
        g_dest[node] = sys.maxint
    g_source[source] = 0
    g_dest[dest] = 0
    
    fq = Queue.PriorityQueue()
    h_source_dest = sg.get_dist(source, dest, h_fn)
    fq.put((h_source_dest, source))
    fq.put((h_source_dest, dest))
    num_queries = []
    while not fq.empty():
        min_f, current = fq.get()
            
        if current in source_frontier:
            res = expand_frontier(sg, current, 
                                  source_frontier, dest_frontier, fq,
                                  g_source, g_dest, 
                                  came_from_source, came_from_dest,
                                  closed, num_queries,
                                  g_fn, h_fn
                                 )
            
        elif current in dest_frontier:
            res = expand_frontier(sg, current, 
                                  dest_frontier, source_frontier, fq,
                                  g_dest, g_source,
                                  came_from_dest, came_from_source,
                                  closed, num_queries,
                                  g_fn, h_fn
                                 )
        closed.add(current)
            
        if res != (None, None):
            path, path_length = res
            if dest == path[0] and source == path[-1]:
                path.reverse()
            return (path, path_length, sum(num_queries))    

In [39]:
shortest_path, path_length, nq = frontier_search(sg, 5555, 2222, set())

Path found at: 8 5555


In [40]:
shortest_path

[5555, 8, 13239, 1266, 2222]

In [41]:
path_length

4

In [42]:
nq

82

In [44]:
path, length, gnf, nq = astar_distance_based(sg, 5555, 2222, set(), "train", g_fn=dists.unweighted)

Destination Reached
Number of Queries made: 146
